# Linear Regression

## Overview

In this lab, you will leverage some of your existing skills to extract NetFlow data from the repository on the VM, plot that data, and then attempt to fit that data using several different regression methods.

## Goals

By the end of this lab, you should be able to:

 * Better extract remote data and import it into a Python process
 * Perform a Linear Regression
 * Perform regressions using higher order functions
 * Plot and analyze the outputs of your regressions
 * Measure the accuracy of these regressions
 
## Estimated Time: 45 - 60 minutes

# Obtaining the Data
This lab will leverage your existing knowledge of numpy, matplotlib, and the SSH class that we created on Day 2.

Our first goal is to successfully read NetFlow statistics that tell us the number of bytes per hour that pass over the network.  Once we have this data, we want to graph it.  Once we know what the data looks like and are satisfied that we have imported it correctly, we will create a forecasting function using regression analysis.  To attempt to measure how well our forecasting function works, we will only use a subset of the ground-truth data for the regression.  This will allow us to measure how well the function forecasts other known data.

# <img src="../images/task.png" width=20 height=20> Task 1.1

Please use the following cell to:
 * Import the *numpy* library with the typical `np` alias
 * Import the *pyplot* package from *matplotlib* using the typical `plt` alias
 * Recreate (or copy) the SSH class that we developed in the final lab during Day 2

Assuming that our libraries are loaded and our SSH class is ready to be used, let's just test things out.

# <img src="../images/task.png" width=20 height=20> Task 1.2

In the following cell, use the SSH class to execute the following command against the NetFlow repository.  Capture the result in a variable named `data`.  As a reminder, the host and credentials are below:

- **Hostname:** 595data.enclaveforensics.com
- **Username:** sans
- **Password:** training

**Remote Command to Execute:**
```
rwfilter --type=all --start-date=2020/01/01 --end-date=2021/03/01 --proto=0-255 \
    --pass=stdout | rwcount --no-titles --delimited=, --bin-size=86400
```

# <img src="../images/task.png" width=20 height=20> Task 1.3

Before continuing, we should make sure that `data` contains what we think it should.  This allows us to verify that our command has executed correctly and the data has been returned.  This also allows us to inspect the structure of the data so that we can prepare to parse it.

Print the value of `data` that was returned.  Verify that it appears to be flow data giving the statistics over a period of 1 day periods from around January 1, 2020, through February 20, 2021.

Now that our data has been returned, we can get to work preprocessing it.  What we appear to have are a series of comma separated fields on lines with the following structure:

|     Date           | Records/Flows | Bytes      | Packets         |
|:------------------:|:-------------:|:----------:|:---------------:|
|2019/05/03T12:55:00|1231.73|11576472.78|36115.26|
|2019/05/03T13:00:00|5100.00|28034694.10|67292.59|
|2019/05/03T13:05:00|4190.39|19801249.46|52100.03|
|2019/05/03T13:10:00|11359.62|9700616.61|38636.73|

What we need to do is turn these into a numpy array.  Before we do, let's think about what it is we're trying to do and whether there is any data that we can drop.  Right now we are working on producing a graph representing the amount of data (bytes).  Could we easily imagine wanting to see a graph of the number of flows or the number of packets?  Certainly.  Given this, we probably want to pull out that data as well.

On the other hand, does the timestamp matter so much?  If we are trying to create a dashboard that shows the amount of data at a specific time, then yes.  However, that's not our task.  What we're trying to do is look at the data over time in one day increments.  The actual time or date doesn't matter much at this point, and may never matter.  The only thing that is relevant is that the data is presented in the correct order.

Given these facts, we would like to keep the last three fields on each line and discard the first.

# Convert and Visualize the Data

# <img src="../images/task.png" width=20 height=20> Task 1.4

Write Python code that extracts the number of flows, bytes, and packets from each line of data, aggregating these lines into a numpy array named `stats` that contains data of type `np.float32`.  To help you verify success, `stats` should have the shape `(331,3)` when you are done.

# <img src="../images/task.png" width=20 height=20> Task 1.5

Now that we have turned the data into a numpy array, we are ready to have a look at the data.  Please write a Python function that will allow you to pass in a set of values in a numpy array.  The function should use matplotlib to draw a visualization of this data.

Once this function is written, generate plots of the flows, bytes, and packets present in the `stats` array.

Now that we have our data in hand and have the ability to do some basic visualization, let's see if we can generate a function to try to fit this data.  Even as we begin, you should have a strong intuition that a Linear Regression is probably a *terrible* way to forecast this specific dataset.  After all, this type of function will provide us a function for a straight line that attempts to predict the behavior of our data.  Our data does not show any strong trend over time, at least not upon first inspection.  Part of this is a result of the scale of the y-axis coupled with the very large outlier.  If you look carefully, you can see that there does appear to be a gradual rise overall from left to right.  Let's perform the Linear Regression because there are some very important lessons to learn, even with this data.

Before we do this let's just take a brief detour back to statistics for a few minutes...

# Moving Average and Bollinger Bands&reg;

This section of the course is primarily focused on deep learning neural networks.  However, we don't want to overlook opportunities to apply knowledge we already have or to add a few useful ideas to what we know.

Looking at this data, there are various interesting things about it.  One of the most interesting is that the major spikes do not correspond with one another between the connections and packets or bytes plots.  Your instructor will no doubt discuss this, or you can listen to the OnDemand recording that discusses and makes observations about this.  However, looking at the data, we can see that there are large spikes that occur.  Our basic introduction to statistics should inform us that there must be a way to find those spikes in a reasonable way without creating some kind of hard boundary condition that is inflexible.

One such way to approach this is the use of a *Moving Average.*. A moving average is calculated in the same way as a typical mean calculation, except that we use a sliding window to generate averages across the data.  This would allow us to generate an average that looks at thirty days at a time, for example.

# <img src="../images/task.png" width=20 height=20> Side Task 1

Examine the code in the following cell and then execute it.  This will move our data into a Pandas dataframe and then use the `rolling()` method to calculate moving averages of our data.  Finally, it plots that average along with our existing data:

In [ ]:
import pandas as pd

df = pd.DataFrame()
df['flows'] = np.array(stats[:,0])
df['moving'] = df['flows'].rolling(30).mean()

df.plot()

A Bollinger Band&reg; is, surprisingly, a registered trademark for an exceptionally simple statistical tool.  Of course, the name makes it sound complicated, but it is simply plotting two standard deviations above and below the moving average.


# <img src="../images/task.png" width=20 height=20> Side Task 2

Examine the code in the following cell and then execute it.  This will generate bollinger bands and plot them alongside the data and moving average values.


In [ ]:
df['b+'] = df['moving'] + df['flows'].rolling(30).std() * 2
df['b-'] = df['moving'] - df['flows'].rolling(30).std() * 2

plt.figure(figsize=(12,3))
plt.plot(df['flows'], color='k')
plt.plot(df['moving'], color='r')
plt.plot(df['b+'], color='g')
plt.plot(df['b-'], color='g')
plt.fill_between(range(len(df['b-'])),df['b-'], df['b+'], color='lightgrey')
plt.grid()
plt.show()

Looking at the bands that are created, you can see that there are a few places (or days, really) where the data rises above the Bollinger band value.  Around day 75, it nearly falls below the lower band.  It again flirts with dropping below around day 160, but never does.

How is this useful?  Using this type of technique can provide us with a useful tool for automatically finding a threshold value (the bands).  When whatever data is being monitored crosses one of the thresholds, an alarm can be generated.

# Perform a Linear Regression

Numpy provides the `polyfit()` function, which can be used to generate a regression to describe a function of any order.  Here is the documentation from the Numpy documentation:

> numpy.polyfit(x, y, deg, rcond=None, full=False, w=None, cov=False)
>
>Least squares polynomial fit.
>
>Fit a polynomial p(x) = p[0] * x**deg + ... + p[deg] of degree deg to points (x, y). Returns a vector of coefficients p that minimises the squared error in the order deg, deg-1, ... 0.

To use this function, we are required to pass in two arrays containing *x* and *y* values along with the *degree* or *order* of the equation.  Since we are trying to fit a line, this is a first order equation, or an equation of degree 1.

Our *x* and *y* values are fairly straightforward.  We already have all of the data for flows, bytes, and packets in our `stats` array.  The *x* coordinates would be an array that is a range of values from zero through the number of elements in the `stats` array.

`polyfit()` returns a list that contains the coefficients that could be used to build a function to produce the mapping from *x* to *y*.

**Important:** We are not going to generate our fit using all of the data.  We are going to "hold out" or reserve the last 30 values from each set of data so that we have something to measure our regressions by later, trying to validate how well they predict reality.

# <img src="../images/task.png" width=20 height=20> Task 1.6

Use the `polyfit()` function to fit the data in `stats`.  Obtain the values for the coefficients for the functions that fit the flows, bytes, and packets for all ***except*** the last thirty values of each.

Now that we have the coefficients for our equation, we need to turn them into functions.  The numpy `poly1d()` function will accept the list of coefficients that we obtained from `polyfit()` and convert it into a Python function that we may call with an *x* value, returning the value of *y*.

# <img src="../images/task.png" width=20 height=20> Task 1.7

Use the next cell to modify your plotting function so that you can plot the data against the result of the regression.  To do this, you will need to use the numpy `poly1d()` function to convert each of the results previously obtained from `polyfit()` into functions.

When revising your function, it would be best to revise it in such a way that you can pass a list of functions to it.  If this list is present, the function will be used to generate values using the range of *x* values already generated within the function.  For extra credit, each function should be plotted using a different color and/or a different line style.

Once you have completed your function, plot each set of data (flows, bytes, and packets) with the corresponding result of the linear regression for each.

***Note:*** You can plot the entire range of data, *including the holdout data!*  We reserved the data so that we would have other data that can be used to measure the performance of the regression result. However, when plotting the data there is no need to exclude this data.

Looking at these lines, they do seem to capture the general overall pattern in the data.  How good are they *really*, though?

# <img src="../images/task.png" width=20 height=20> Task 1.8

Write a function that returns the mean squared error by comparing two input arrays.  Once you have written the function, print the error by comparing each type of data in `stats` with the results of the respective regression functions.

For validation, loss values that you obtain should be:

```
Flows: 1442916340.648097
Bytes: 4.544266163130101e+21
Packets: 2919638274061851.5
```

# <img src="../images/task.png" width=20 height=20> Task 1.9

Now that you have MSE function and can see how the functions do overall, let's compare the overall error to the error when compared to our hold out data.

Rerun the error analysis, this time generating the overall MSE and comparing it to the MSE of only the hold out data for each set of data in `stats`.  Since we are using a mean squared error, both of the values should be in the same scale (as opposed to using the sum of squared errors, which would not).  Let us define "Better" or "Acceptable" to mean that the mean squared error of our hold out data is *less than* the mean squared error of the data overall.

Generate output that indicates whether the MSE of the hold out data is better than that of the data overall.


# Higher Degree Functions

Let's generate some additional higher degree polynomials and see how well they fit our data.  To do this, we simply need to send a degree value higher than 1 to the `polyfit()` function.

# <img src="../images/task.png" width=20 height=20> Task 1.10

Use the `polyfit()` function to generate 2nd, 3rd, 4th, 8th, 11th, and 14th order equations that fit each of the data types in `stats`.  Generate a single graph that compares all of the functions that you obtain, including the 1st order equation.

***Hint:*** You will almost *certainly* need to investigate adding limits to your y-axis or the range of the graphs will make it impossible to see what the fitted functions are doing.  You can do this using something like `plt.ylim(top=y.max())` and `plt.ylim(bottom=y.min())` where *y* is the data that you are plotting in an numpy array.  This will require you to revise your plotting function.

Some of these functions do seem to be a better overall approximation of the data... but some of them seem to be off-the-charts wrong!  Look at your graphs very carefully.  Can you see visually where things seem to go off the rails for the 11th and 14th order equations?  Doesn't it appear to be right around 330 or so?

That number seems very familiar... Can you work out a reason why that number might be where things go wrongly?  For a hint, run the next cell:

In [ ]:
stats.shape

Hmmm.... So stats has 331 rows... So what?

Do you remember that we generated our regression by holding out the last thirty values?  No wonder things go wrong right around 330!  That holdout data just made itself relevant!

Still, we should compare the errors.

# <img src="../images/task.png" width=20 height=20> Task 1.11

Measure the overall MSE for each of the seven functions for each of the three statistics.  Compare this to the MSE for the holdout data *only*.  How do each of the regressions compare?

# Conclusion

As simple as this lab is, there are some important outcomes.  First, you should have found it a bit easier to leverage the skills covered earlier in the week to create the necessary code.  You should also find yourself more naturally desiring to refactor your code into reusable elements.  If you have not already done so, you should research how to create your own Python module or package so that you can generate useful utility functions that you find yourself using over and over, preventing yourself from having to rewrite them every time.  For example:

 * The SSH class - Can you see yourself needing to use an SSH connection to query something or retrieve some data at some point in the future?  Can you see yourself needing to do that many, many times?
 * The MSE function - While the scikit library does include a mean squared error function, we find it is better to limit the number of dependencies in our code (not importing dozens of libraries or modules if it can be avoided).  You will also likely develop other loss or error functions over time that are not part of any standard module or library.  Why not start collecting them in your own library now?
 * The plotting function for functions - How many times will you want to render a graph with multiple functions?  Alternatively, when we begin working with TensorFlow, you will very frequently want to plot the accuracy and loss values of your models or to plot evaluation comparisons.  Why not start generating that library of functions now?
 
Another very important takeaway is that of overfitting.  As discussed, overfitting can lead to what amounts to memorization of the data.  While a regression function of the sort that we have generated in this lab doesn't really have a "memory," the idea is still valid.  The regressions that we have done well illustrate the notion of a model that has very low bias but very high variance.  The more complex our model, the higher the variance, leading to spending more time fitting the noise, or irrelevant data.  As expressed in the discussion, our models all perform well *within the training data*, but the higher order functions perform increasingly unwell even though their raw error values are lower in some cases!